# DEAP

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, AvgPool1D, Dense, Conv1D, Flatten, Dropout, Input, BatchNormalization, GlobalMaxPool1D, MaxPool1D, SpatialDropout1D, GlobalAvgPool1D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler                                                      
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix 
from scipy import signal
import pickle as pkl
import gc

import wandb

In [2]:
sampling_rate = 128
window_size = 1280
overlap = 256
channel_len = 32
classes=2

bands = {'delta': [0.5/(sampling_rate/2), 4/(sampling_rate/2)], 'theta': [4/(sampling_rate/2), 8/(sampling_rate/2)], \
         'alpha': [8/(sampling_rate/2), 14/(sampling_rate/2)], 'beta': [14/(sampling_rate/2), 30/(sampling_rate/2)], \
         'gamma': [30/(sampling_rate/2), 50/(sampling_rate/2)], 'all': [0.5/(sampling_rate/2), 50/(sampling_rate/2)]} # FIXED: 75 -> 50

import os

os.environ["CUDA_VISIBLE_DEVICES"]="3"

## Loading Data

In [3]:
def load_data(eeg_band = None):

  eeg_signal = []
  valence = []
  arousal = []
  dominance = []
  # Modified
  subject = {} # FIXED: Add subject dict
  indexes = []

  for i in range(32):
    subject[i] = "{0:02d}".format(i+1)

  for person in range(32):
    print('Person No.' + str(person))
    
    # EEG files address
    address = './eeg_data/DEAP/s'+subject[person]+'.dat'

    with open(address, 'rb') as file:
      data = pkl.load(file, encoding = 'latin1')

    eeg = data['data']
    label = data['labels']
    
    # Assigning classes
    label[label<5.5] = 0
    label[label>=5.5] = 1     

    val = label.T[0] # Valence label
    aro = label.T[1] # Arousal label
    dom = label.T[2] # Dominance label

    del data, label
    

    for i in range(40): # Iterating through 40 vidoes/trials

      sig = eeg[i].T
      v = val[i]
      a = aro[i]
      d = dom[i]

      sig = sig[:, :32] # Considering all 32 EEG channels
      
      num, den = signal.butter(4, bands[eeg_band], 'band') # Butterworth filter of order N = 4
      band_signal = signal.filtfilt(num, den, sig, axis=0)
      sig = band_signal
      del band_signal, num, den
      
      scaler = StandardScaler().fit(sig)
      scaled_sig = scaler.transform(sig)

      del sig
        
      # Segmenting into 10 seconds (1280 timesteps) windows with 2 seconds (256 timesteps) overlap
      start = 0
      while start + window_size < scaled_sig.shape[0]:
        eeg_signal.append(scaled_sig[start:start+window_size, :])

        valence.append(v)
        arousal.append(a)
        dominance.append(d)
        start += overlap # NOTE: Isn't 'window_size - overlap' right?
      # indexes.append((person, len(eeg_signal) - 1))
      del scaled_sig
    
    indexes.append((person, len(eeg_signal) - 1))


  eeg_signal = np.asarray(eeg_signal, dtype = np.float32) # Using 32 bit floating point value to save memory
  valence = np.asarray(valence, dtype = np.int8)
  arousal = np.asarray(arousal, dtype = np.int8)
  dominance = np.asarray(dominance, dtype = np.int8)

  print(valence.shape, valence[valence == 0].shape, valence[valence == 1].shape)
  print(arousal.shape, arousal[arousal == 0].shape, arousal[arousal == 1].shape)
  print(dominance.shape, dominance[dominance == 0].shape, dominance[dominance == 1].shape)

  valence = np_utils.to_categorical(valence)
  arousal = np_utils.to_categorical(arousal)
  dominance = np_utils.to_categorical(dominance)

  print(indexes)
    
  return (eeg_signal, valence, arousal, dominance)

In [4]:
FOLD = 10 # Select 4 trials' data for a test set per 1 fold.
eeg_band = 'beta'#'gamma' # EEG band name

data, valence, arousal, dominance = load_data(eeg_band) # Loading processed data

nb_samples = data.shape[0] # Number of samples
factor = 1080//FOLD # Kth fold by this factor, 1080 samples per one subject
# factor_ho = 7*nb_samples//10

# shuffler = np.random.permutation(nb_samples) # Shuffling data

# data = data[shuffler]
# valence = valence[shuffler]
# arousal = arousal[shuffler]
# dominance = dominance[shuffler]

Person No.0
Person No.1
Person No.2
Person No.3
Person No.4
Person No.5
Person No.6
Person No.7
Person No.8
Person No.9
Person No.10
Person No.11
Person No.12
Person No.13
Person No.14
Person No.15
Person No.16
Person No.17
Person No.18
Person No.19
Person No.20
Person No.21
Person No.22
Person No.23
Person No.24
Person No.25
Person No.26
Person No.27
Person No.28
Person No.29
Person No.30
Person No.31
(34560,) (18711,) (15849,)
(34560,) (17766,) (16794,)
(34560,) (17739,) (16821,)
[(0, 1079), (1, 2159), (2, 3239), (3, 4319), (4, 5399), (5, 6479), (6, 7559), (7, 8639), (8, 9719), (9, 10799), (10, 11879), (11, 12959), (12, 14039), (13, 15119), (14, 16199), (15, 17279), (16, 18359), (17, 19439), (18, 20519), (19, 21599), (20, 22679), (21, 23759), (22, 24839), (23, 25919), (24, 26999), (25, 28079), (26, 29159), (27, 30239), (28, 31319), (29, 32399), (30, 33479), (31, 34559)]


## CNN Model

In [5]:
eeg_input = Input(shape = (window_size, channel_len), name='eeg_input') # Input layer

# CNN model
def get_CNN_design():
  x = Conv1D(filters = 32, kernel_size = 1, strides = 2, padding = 'valid', activation='relu', name='conv1')(eeg_input)
  x = Conv1D(filters = 32, kernel_size = 5, strides = 2, padding = 'valid', activation='relu', name='conv2')(x)
  x = AvgPool1D(pool_size=2, name='avg_pool1')(x)
  x = BatchNormalization(name='batch_norm1')(x)
  x = SpatialDropout1D(rate=0.0625, name = 'spatial_dropout1')(x)
  x = Conv1D(filters = 64, kernel_size = 5, strides = 2, padding = 'valid', activation='relu', name='conv3')(x)
  x = Conv1D(filters = 64, kernel_size = 5, strides = 2, padding = 'valid', activation='relu', name='conv4')(x)
  x = AvgPool1D(pool_size=2, name='avg_pool2')(x)
  x = BatchNormalization(name='batch_norm2')(x)
  x = SpatialDropout1D(rate=0.125, name = 'spatial_dropout2')(x)
  x = Conv1D(filters = 128, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv5')(x)
  x = Conv1D(filters = 128, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv6')(x)
  x = AvgPool1D(pool_size=2, name='avg_pool3')(x)
  x = BatchNormalization(name='batch_norm3')(x)
  x = SpatialDropout1D(rate=0.25, name = 'spatial_dropout3')(x)
  x = Conv1D(filters = 256, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv7')(x)
  x = Conv1D(filters = 256, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv8')(x)
  x = GlobalAvgPool1D(name='global_pool1')(x)
  x = BatchNormalization(name='batch_norm4')(x)
  x = Dropout(0.2)(x)
  x = Dense(64)(x)
  x = Activation('tanh')(x)
  x = Dense(8)(x)
  x = Activation('tanh')(x)
  x = Dropout(0.1)(x)
  return x

def get_model():
  x = get_CNN_design()

  out = Dense(classes, activation='softmax', name = 'output')(x) # Output layer

  model = Model(inputs=eeg_input, outputs=out) # Creating a model instance
  
  adam = Adam(lr=1e-3,decay=1e-5) # Adam optimizer
  
  model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['categorical_accuracy']) # Compiling model
  model.summary()
  
  return model

In [6]:
wandb.init(project="emosync",)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gywls0724 (emosync). Use `wandb login --relogin` to force relogin


## Training

### Valence

In [7]:
val_res = {'accuracy': [], 'confusion_matrix': []}


gc.collect() # Garbage collecter
for i in range(FOLD):
  print('Fold: '+str(i))

  # X_train = data[0 : factor_ho] # Training data
  # X_test = data[factor_ho : nb_samples] # Testing data
  # val_train = valence[0 : factor_ho] # Valence training labels
  # val_test = valence[factor_ho : nb_samples] # Valence testing labels
  
  # Shuffle -> similar results
  X_train = []
  X_test = []
  val_train = []
  val_test = []
  for p in range(32):
    offset = p * 1080
    block_end = (p+1) * 1080
    X_train.append(np.concatenate((data[offset : offset + i*factor], data[offset + (i+1)*factor : block_end])))
    X_test.append(data[offset + i*factor : offset + (i+1)*factor])
    val_train.append(np.concatenate((valence[offset : offset + i*factor], valence[offset + (i+1)*factor : block_end])))
    val_test.append(valence[offset + i*factor : offset + (i+1)*factor])

  X_train = np.concatenate(tuple(X_train)) # Training data
  shuffler = np.random.permutation(len(X_train)) # Shuffling data
  X_train = X_train[shuffler]
  val_train = np.concatenate(tuple(val_train)) # Valence training labels
  val_train = val_train[shuffler]

  X_test = np.concatenate(tuple(X_test)) # Testing data
  shuffler = np.random.permutation(len(X_test)) # Shuffling data
  X_test = X_test[shuffler]
  val_test = np.concatenate(tuple(val_test)) # Valence testing labels
  val_test = val_test[shuffler]

  gc.collect() # Garbage collecter
  model = get_model()
  model.fit(X_train, val_train, epochs = 100, batch_size = 1024, shuffle = True, callbacks=[wandb.keras.WandbCallback(labels=["Low", "High"])])

  acc = model.evaluate(X_test, val_test)
  print(acc)

  val_res['accuracy'].append(acc)

  pred = model.predict(X_test)
  val_res['confusion_matrix'].append(confusion_matrix(val_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter


# Dumping valence results
file = './eeg_data/DEAP_k-foldCV_unseen_trials_results/DEAP_valence_' + eeg_band + '.pkl'

with open(file, 'wb') as f:
  pkl.dump(val_res, f)


Fold: 0


/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            10304 

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            1030

...

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            1030

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            1030

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            1030

In [8]:

# # Dumping valence results
# file = './eeg_data/DEAP_k-foldCV_unseen_trials_results/DEAP_valence_' + eeg_band + '.pkl'

# with open(file, 'wb') as f:
#   pkl.dump(val_res, f)

for i in val_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places

36.0
44.82
48.87
44.1
45.49
50.72
47.54
29.63
60.21
53.94


In [9]:
for i in val_res['confusion_matrix']:
  print(i)

[[ 431  109]
 [2103  813]]
[[ 743  229]
 [1678  806]]
[[ 560  412]
 [1355 1129]]
[[ 815  292]
 [1640  709]]
[[ 494  235]
 [1649 1078]]
[[1429 1406]
 [ 297  324]]
[[1059 1371]
 [ 442  584]]
[[ 597 2373]
 [  59  427]]
[[1920 1239]
 [ 136  161]]
[[1611 1386]
 [ 206  253]]


### Arousal

In [10]:
aro_res = {'accuracy': [], 'confusion_matrix': []}

gc.collect() # Garbage collecter
for i in range(FOLD):
  print('Fold: '+str(i))
  
  # Shuffle -> similar results
  X_train = []
  X_test = []
  aro_train = []
  aro_test = []
  for p in range(32):
    offset = p * 1080
    block_end = (p+1) * 1080
    X_train.append(np.concatenate((data[offset : offset + i*factor], data[offset + (i+1)*factor : block_end])))
    X_test.append(data[offset + i*factor : offset + (i+1)*factor])
    aro_train.append(np.concatenate((arousal[offset : offset + i*factor], arousal[offset + (i+1)*factor : block_end])))
    aro_test.append(arousal[offset + i*factor : offset + (i+1)*factor])

  X_train = np.concatenate(tuple(X_train)) # Training data
  shuffler = np.random.permutation(len(X_train)) # Shuffling data
  X_train = X_train[shuffler]
  aro_train = np.concatenate(tuple(aro_train)) # Valence training labels
  aro_train = aro_train[shuffler]

  X_test = np.concatenate(tuple(X_test)) # Testing data
  shuffler = np.random.permutation(len(X_test)) # Shuffling data
  X_test = X_test[shuffler]
  aro_test = np.concatenate(tuple(aro_test)) # Valence testing labels
  aro_test = aro_test[shuffler]

  gc.collect() # Garbage collecter
  model = get_model()
  model.fit(X_train, aro_train, epochs = 100, batch_size = 1024, shuffle = True)

  acc = model.evaluate(X_test, aro_test)
  print(acc)

  aro_res['accuracy'].append(acc)

  pred = model.predict(X_test)
  aro_res['confusion_matrix'].append(confusion_matrix(aro_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter


# Dumping arousal results
file = './eeg_data/DEAP_k-foldCV_unseen_trials_results/DEAP_arousal_' + eeg_band + '.pkl'

with open(file, 'wb') as f:
  pkl.dump(aro_res, f)


Fold: 0


/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

...

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

In [11]:
for i in aro_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places

53.39
59.26
55.79
52.03
56.57
59.61
58.56
59.43
45.46
53.1


In [ ]:
for i in aro_res['confusion_matrix']:
  print(i)

[[539 147]
 [357  77]]
[[278 352]
 [235 255]]
[[306 289]
 [223 302]]
[[349 169]
 [434 168]]
[[321 155]
 [448 196]]
[[233 271]
 [239 377]]
[[309 307]
 [223 281]]
[[433 148]
 [375 164]]


### Dominance (Ignored)
This code is not working. 

In [ ]:
dom_res = {'accuracy':[], 'confusion_matrix':[]}

gc.collect() # Garbage collecter
for i in range(FOLD):
  print('Fold: '+str(i))

  X_train = []
  X_test = []
  dom_train = []
  dom_test = []
  for p in range(32):
    offset = p * 1080
    X_train.append(np.concatenate((data[offset : offset + i*factor], data[offset + (i+1)*factor : nb_samples])))
    X_test.append(data[offset + i*factor : offset + (i+1)*factor])
    dom_train.append(np.concatenate((dominance[offset : offset + i*factor], dominance[offset + (i+1)*factor : nb_samples])))
    dom_test.append(dominance[offset + i*factor : offset + (i+1)*factor])

  X_train = np.concatenate(tuple(X_train)) # Training data
  shuffler = np.random.permutation(len(X_train)) # Shuffling data
  X_train = X_train[shuffler]

  X_test = np.concatenate(tuple(X_test)) # Testing data
  shuffler = np.random.permutation(len(X_test)) # Shuffling data
  X_test = X_test[shuffler]

  dom_train = np.concatenate(tuple(dom_train)) # Valence training labels
  shuffler = np.random.permutation(len(dom_train)) # Shuffling data
  dom_train = dom_train[shuffler]

  dom_test = np.concatenate(tuple(dom_test)) # Valence testing labels
  shuffler = np.random.permutation(len(dom_test)) # Shuffling data
  dom_test = dom_test[shuffler]
  
  gc.collect() # Garbage collecter
  model = get_model()
  model.fit(X_train, dom_train, epochs = 100, batch_size = 1024, shuffle = True)

  acc = model.evaluate(X_test, dom_test)
  print(acc)
    
  dom_res['accuracy'].append(acc)

  pred = model.predict(X_test)
  dom_res['confusion_matrix'].append(confusion_matrix(dom_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter


# Dumping dominance results
file = './eeg_data/DEAP_k-foldCV_unseen_trials_results/DEAP_dominance_' + eeg_band + '.pkl'

with open(file, 'wb') as f:
  pkl.dump(dom_res, f)

Fold: 0


/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

...

Model: "model_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

In [ ]:
for i in dom_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places

43.21
48.66
50.27
48.21
44.11
50.09
38.21
54.64


In [ ]:
for i in dom_res['confusion_matrix']:
  print(i)

[[298 339]
 [297 186]]
[[345 285]
 [290 200]]
[[316 349]
 [208 247]]
[[322 350]
 [230 218]]
[[136 494]
 [132 358]]
[[166 387]
 [172 395]]
[[235  80]
 [612 193]]
[[287 210]
 [298 325]]
